# Task Description

1 run = 128 trials = 96 Go trials (75%) + 32 Stop trials (25%)

32 Stop trials = 8 trials per each staircase X 4 staircases


# Bayesian Parametric Estimation of Stop-Signal Reaction Time Distributions

Dora Matzke, Conor V. Dolan, Gordon D. Logan, Scott D. Brown and Eric–Jan Wagenmakers

The integration method assumes that SSRT is constant. SSRTs are
estimated from the observed go RT distribution and the P(respond | stop–signal) by finding the point (i.e., SSRT + SSD) at which the integral of the go RT distribution equals the P(respond | stop–signal)

the integration method involves __deriving the time point at which the internal response to the stop–signal occurs and subtracting SSD to obtain the SSRT__. 

__In practice, the following procedure is used: Go RTs are collapsed into a single distribution and are rank ordered. Subsequently, the nth go RT is selected, where n is obtained by multiplying the number of go RTs by the P(respond | stop–signal) at a given SSD. Lastly, the SSD is subtracted to arrive at the SSRT. The integration method yields SSRT estimates for each SSD. As estimated SSRTs tend to decrease with increasing SSD (Logan & Burkell, 1986; Logan & Cowan, 1984), SSRTs at different SSDs are often averaged to yield a summary score for each participant.__

The integration method has several drawbacks. It assumes that SSRT is constant, an assumption that is certainly incorrect. Moreover, the integration method requires a relatively large number of observations to produce accurate estimates of average SSRT. Researchers are advised to present participants with at least 900 go trials and 60 stop–signal trials on each of five different SSDs (Band, Van Der Molen, & Logan, 2003).


# Fictitious Inhibitory Differences: How Skewness and Slowing Distort the Estimation of Stopping Latencies

Frederick Verbruggen, Christopher D. Chambers, and Gordon D. Logan 2013

SSRT is estimated according to the independent-race model (Logan, 1994; Logan & Cowan, 1984; Verbruggen & Logan, 2009a): Performance in the stop task can be modeled as a race between a go process, which is triggered by the presentation of the go stimulus, and a stop process, which is triggered by the presentation of a stop signal. The stop signal occurs after a variable interval, the stop-signal delay (SSD). If the go process finishes before the stop process (i.e., when RT < (SSRT + SSD)), then response inhibition is unsuccessful and a response is executed; if the stop process finishes before the go process (i.e., when RT > (SSRT + SSD)), then the response is correctly withheld. 

In the integration method, the point at which the stop process finishes is estimated by __integrating the RT distribution and finding the point at which the integral equals the probability of responding, p(respond|signal), for a specific delay. SSRT is then calculated by subtracting SSD from the finishing time.__

__The integration method assumes that the finishing time of the stop process corresponds to the nth RT, with n equal to the number of RTs in the RT distribution multiplied by the overall p(respond|signal) (Logan, 1981); SSRT can then be estimated by subtracting the mean SSD from the nth RT (e.g., Ridderinkhof, Band, & Logan, 1999; Verbrug- gen, Liefooghe, & Vandierendonck, 2004).__

For the first set of simulations, we estimated SSRT __over all blocks__ using the mean method (SSRT = mean RT – mean SSD) and __the integration method (SSRT = nth RT – mean SSD)__. For the second and third set, we also __estimated SSRT for each block separately using the integration method and then took the average of these four block estimates.__ Trials with an __RT higher than 2,000 were considered to be missed responses__ (in real experiments, there is always a response deadline around this value). These missed trials were excluded when we estimated SSRT using the mean method; for the integration methods, RT for missed responses was set to 2,000.

The integration method fared better in the first set of simulations: There was a trend to underestimate SSRT slightly (approximately 4 ms), but there were no obvious group differences caused by changes in the shape of the RT distribution. This is consistent with a recent reliability analysis that used split-half reliability measures (Congdon et al., 2012). However, the second and third set of simulations showed that the small underestimation bias for the integration method became more pronounced when there is gradual slowing of RTs across blocks. This underestimation bias may explain the previously observed negative correlations between SSRT and response slowing (e.g., Jahfari et al., 2010; Leotti & Wager, 2010). Thus, we have demonstrated that the experiment-wide integra- tion method results in reliable and unbiased estimates unless subjects slow their RT gradually.

The second and third set of simulations show that a __block-based version of the integration method is less susceptible to bias from response slowing__. When SSRT was estimated for each block separately (number of no-signal trials per block = 45; number of signal trials per block = 15) and then averaged, we obtained a reliable and unbiased SSRT even when there was substantial response slowing. Additional analyses (Figs. S5–S6 in the Supplemental Material) suggest that approximately 40 to 80 trials are required per block (25% of which are signal trials). If there are fewer trials, the estimates become too noisy; if there are more trials, the underestimation bias starts to emerge. We recommend that there are at least 50 signals in total. Thus, we feel that researchers should estimate SSRT for each block separately when strategic slowing is observed and subjects cannot be excluded.

<img src="../files/VerbruggenChambersLogan2013.png">

In [2]:
import pandas as pd
import os
import numpy as np
from glob import glob
from numpy import nan
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
bids_dir = '/Shared/oleary/functional/UICL/BIDS/'
txt_dir = '/raid0/homes/crcshare/people/Dan_Oleary/UI_College_Life/fMRI_behav_data'

sublist = glob(os.path.join(bids_dir, 'sub-*'))
sublist = [lb.split('-')[1] for lb in sublist]
runs = ['1','2']

# Create blank dataframe

finaldata_collector = pd.DataFrame()

# Start for loop

for sub in sublist:
    for run in runs:
        txtfile = os.path.join(txt_dir, sub, sub + '_ST_' + run + '.txt')
        blffile = os.path.join(txt_dir, sub, sub + '_ST_' + run + '.blf')
        
        # GETTING SSD INFO
        ssd_collector = []
        collect = False
        with open(txtfile, 'r') as ssd:
            for line in ssd:
                if line.startswith('Subject'):
                    collect=True
                if line.startswith('Event Type'):
                    collect=False
                if collect:
                    ssd_collector.append(line)           
        ssdlist = [str.strip('\n').split('\t') for str in ssd_collector]
        headers = ssdlist.pop(0)
        # import dataframe in pandas, exclude the first row
        ssd_data = pd.DataFrame(ssdlist, columns=headers)
        ssd_data = ssd_data.drop([0])
        # exclude trials in which respond is faster than SSD
        for idx in ssd_data.index:
            if ssd_data.loc[idx]['Event Type'] == 'Picture':
                if ssd_data.loc[idx+1]['Event Type'] != 'Sound':
                    ssd_data.drop(idx, inplace=True) 
        # for stop trials, get onset time of go stim
        stim_onset = ssd_data.loc[(ssd_data['Code'] == 'inhib_go:lt') | (ssd_data['Code'] == 'inhib_go:rt') , 'Time']
        # for stop trials, get onset time of stop stim
        buzzer_onset = ssd_data.loc[(ssd_data['Code'] == 'inhibit'), 'Time']
        # forcing those onsets to be interrger
        stim_onset = pd.to_numeric(stim_onset, errors='coerce')
        buzzer_onset = pd.to_numeric(buzzer_onset, errors='coerce')
        # creating a variable called 'ssd' storing all SSDs, then convert it to ms)
        # SSD = Buzzer onset - Arrow onset
        ssd = np.subtract(buzzer_onset, stim_onset)
        ssd = ssd/10
        
        # GETTING STAIRCASE INFO

        stair_collector = []
        collect = False
        with open(blffile, 'r') as stair:
            for line in stair:
                if line.startswith('rest'):
                    collect=True
                if line.startswith('StaircaseStart:'):
                    collect=False
                if collect:
                    stair_collector.append(line)

        stairlist = [str.strip('\n').split('\t') for str in stair_collector]

        # import dataframe in pandas
        stair_data = pd.DataFrame(stairlist)
        # dropping all unnecessary rows, so that the dataframe only 
        # include information regarding the 4 staircases
        stair_data = stair_data[stair_data[1].str.contains("0|Right|Left") == False]
        # making a variable storing the order in which each staircase was presented in a run
        staircase = stair_data[1]
        
        
        # GETTING GO RT 

        rt_collector = []
        collect = False
        with open(txtfile, 'r') as rt:
            for line in rt:
                if line.startswith('Event Type'):
                    collect=True
                if collect:
                    rt_collector.append(line)

        rtlist = [str.strip('\n').split('\t') for str in rt_collector]
        headers = rtlist.pop(0) 


        # import dataframe in pandas, exclude the first row
        rt_data = pd.DataFrame(rtlist, columns=headers)
        rt_data = rt_data.drop([0])

        # identify go and stop trials
        rt_data['Go'] = np.where(rt_data.Code == 'go:rt', 'go',np.where(rt_data.Code == 'go:lt','go', ''))
        rt_data['Stop'] = np.where(rt_data.Code == 'inhib_go:rt', 'stop',np.where(rt_data.Code == 'inhib_go:lt','stop', ''))

        # Go trials categories: 
            # correct go (press correct button when there was no buzzer) 
            # incorrect go (press wrong button when there was no buzzer)
            # missed go (did not press button when there was no buzzer)

        # Stop trials categories: 
            # correct stop (suppress pressing button when there was buzzer) 
            # incorrect stop (press correct button when there was buzzer)
            # failed stop (press wrong button when there was buzzer)

        rt_data['trial_type'] = np.where((rt_data.Go == 'go') & (rt_data.Type == 'hit'), 'CorGo',
                                    np.where((rt_data.Go == 'go') & (rt_data.Type == 'incorrect'), 'IncGo',
                                    np.where((rt_data.Go == 'go') & (rt_data.Type == 'miss'), 'MissGo',
                                    np.where((rt_data.Stop == 'stop') & (rt_data.Type == 'miss'), 'CorStop',
                                    np.where((rt_data.Stop == 'stop') & (rt_data.Type == 'hit'), 'IncStop',
                                    np.where((rt_data.Stop == 'stop') & (rt_data.Type == 'incorrect'), 'FailStop',''
                                   ))))))

        # converting values in the  RT column to numeric then calculate the correct RT (ms) from raw data
        rt_data['RT'] = pd.to_numeric(rt_data['RT'])
        rt_data['RT'] = (rt_data['RT'] / 10)

        # for stop trials only, extract trial type, then stored this as a df named 'stop'
        stop = rt_data.loc[(rt_data['Stop'] == 'stop'), 'trial_type']

        # for CorGo trials only, extract trial-by-trial RT, then stored this as a df named 'go_rt'
        go_rt = rt_data.loc[(rt_data['trial_type'] == 'CorGo'), 'RT']
        # calculate mean Go RT of correct go trials
        meanGoRT = go_rt.mean()
        
        # SSRT CALCULATION
        
        # CALCULATE SSRT STEP 1: making a dataframe named 'stop_trials' that included all information about 
        # SSDs, stair cases and trial types OF STOP TRIALS ONLY in a run
        # dropping row and column index of SSD, stair case, and trial types
        ssd.reset_index(drop=True, inplace=True)
        ssd.columns = range(ssd.shape[0])
        staircase.reset_index(drop=True, inplace=True)
        staircase.columns = range(staircase.shape[0])
        stop.reset_index(drop=True, inplace=True)
        stop.columns = range(stop.shape[0])
        # then do concatnation
        stop_trials = pd.concat([ssd, staircase, stop], axis=1, ignore_index=True)
        stop_trials = stop_trials.dropna()
        
        # CALCULATE SSRT STEP 2: calculate P(respond | stop–signal) for each SSD
        # Step 2A: counting number of incorrect stop for each staircase
        n_first_stair_incstop = len (stop_trials[(stop_trials[1].str.contains("2|3|4") == False)  
                                                 & (stop_trials[2].str.contains("IncStop") == True)])
        n_second_stair_incstop = len (stop_trials[(stop_trials[1].str.contains("1|3|4") == False)  
                                                 & (stop_trials[2].str.contains("IncStop") == True)])
        n_third_stair_incstop = len (stop_trials[(stop_trials[1].str.contains("1|2|4") == False)  
                                                 & (stop_trials[2].str.contains("IncStop") == True)])
        n_fourth_stair_incstop = len (stop_trials[(stop_trials[1].str.contains("1|2|3") == False)  
                                                 & (stop_trials[2].str.contains("IncStop") == True)])
                
        # Step 2B: calculate P(respond | stop–signal)
        first_stair_p =  n_first_stair_incstop / 8 * 100
        second_stair_p =  n_second_stair_incstop / 8 * 100
        third_stair_p =  n_third_stair_incstop / 8 * 100
        fourth_stair_p =  n_fourth_stair_incstop / 8 * 100
        
        
        # CALCULATE SSRT STEP 2: find the nth RT 
        GoNRT1 = np.percentile(go_rt, first_stair_p, interpolation = 'midpoint')
        GoNRT2 = np.percentile(go_rt, second_stair_p, interpolation = 'midpoint')
        GoNRT3 = np.percentile(go_rt, third_stair_p, interpolation = 'midpoint')
        GoNRT4 = np.percentile(go_rt, fourth_stair_p, interpolation = 'midpoint')
        
        # STEP 3: Calculate SSRT
        # STEP 3A: Create dataframe of each staircase
            # First, creating a df called 'first_stair' that include stop trials of the 1st staircase
            # remember that 'stop_trials' is the dataframe that include the information about 
            # SSD, stair case, stop trial types (CorStop, IncStop, FailStop)
        first_stair = stop_trials[(stop_trials[1].str.contains("2|3|4") == False)]
        second_stair = stop_trials[(stop_trials[1].str.contains("1|3|4") == False)]
        third_stair = stop_trials[(stop_trials[1].str.contains("1|2|4") == False)]
        fourth_stair = stop_trials[(stop_trials[1].str.contains("1|2|3") == False)]
        
        # Then, calculate the mean SSD across all SSDs of one staircase
        SSD1 = first_stair[0].mean()
        SSD2 = second_stair[0].mean()
        SSD3 = third_stair[0].mean()
        SSD4 = fourth_stair[0].mean()
        
        # STEP 3B: SSRT = nth RT - mean SSD 
        SSRT1 = GoNRT1 - SSD1
        SSRT2 = GoNRT2 - SSD2
        SSRT3 = GoNRT3 - SSD3
        SSRT4 = GoNRT4 - SSD4
        
        # STEP 4: SSRTs at different SSDs are often averaged to yield a summary score for each participant
        meanSSRT = (SSRT1 + SSRT2 + SSRT3 + SSRT4) / 4
        meanNGoRT = (GoNRT1 + GoNRT2 + GoNRT3 + GoNRT4) / 4
        meanSSD = (SSD1 + SSD2 + SSD3 + SSD4) / 4
        
        # CALCULATING GO ACCURACY
        
        # counting the number of correct go trials
        N_CorGo = len(rt_data[(rt_data['trial_type'].str.contains("CorGo") == True)])
        # counting the number of total go trials
        N_Go = 96
        # Accuracy Go trials
        Accuracy_Go = N_CorGo / N_Go * 100
        
        # CALCULATING STOP ACCURACY
        
        # counting the number of correct Stop trials
        N_CorStop = len(rt_data[(rt_data['trial_type'].str.contains("CorStop") == True)])
        # counting the number of total Stop trials
        N_Stop = 32
        # Accuracy Stop trials
        Accuracy_Stop = N_CorStop / N_Stop * 100
        
        # CALCULATING STOP ACCURACY FOR EACH STAIR CASE
        # then, count the number of CorStop trials of the 1st staircase
        n_first_stair_corstop = len (stop_trials[(stop_trials[1].str.contains("2|3|4") == False)  
                                                 & (stop_trials[2].str.contains("CorStop") == True)])
        Accuracy_Stop_1ststair = n_first_stair_corstop / 8 * 100 
        # then, count the number of CorStop trials of the 2nd staircase
        n_second_stair_corstop = len (stop_trials[(stop_trials[1].str.contains("1|3|4") == False)  
                                                  & (stop_trials[2].str.contains("CorStop") == True)])
        Accuracy_Stop_2ndstair = n_second_stair_corstop / 8 * 100 
        # then, count the number of CorStop trials of the 3rd staircase
        n_third_stair_corstop = len (stop_trials[(stop_trials[1].str.contains("1|2|4") == False)  
                                                 & (stop_trials[2].str.contains("CorStop") == True)])
        Accuracy_Stop_3rdstair = n_third_stair_corstop / 8 * 100 
        # then, count the number of CorStop trials of the 4th staircase
        n_fourth_stair_corstop = len (stop_trials[(stop_trials[1].str.contains("1|2|3") == False)  
                                                  & (stop_trials[2].str.contains("CorStop") == True)])
        Accuracy_Stop_4thstair = n_fourth_stair_corstop / 8 * 100
        
        
        N_IncGo = len(rt_data[(rt_data['trial_type'].str.contains("IncGo") == True)])
        Percent_IncGo = N_IncGo / N_Go * 100
        N_MissGo = len(rt_data[(rt_data['trial_type'].str.contains("MissGo") == True)])
        Percent_MissGo = N_MissGo / N_Go * 100
        N_IncStop = len(rt_data[(rt_data['trial_type'].str.contains("IncStop") == True)])
        Percent_IncStop = N_IncStop / N_Stop * 100
        N_FailStop = len(rt_data[(rt_data['trial_type'].str.contains("FailStop") == True)])
        Percent_FailStop = N_FailStop / N_Stop * 100
        
        #Create a Dictionary of series
        d = {'SubID':sub,
             'Run':run,
             'GoNRT1':GoNRT1,
             'GoNRT2':GoNRT2,
             'GoNRT3':GoNRT3,
             'GoNRT4':GoNRT4,
             'meanNGoRT':meanNGoRT,
             'SSD1':SSD1,
             'SSD2':SSD2,
             'SSD3':SSD3,
             'SSD4':SSD4,
             'meanSSD':meanSSD,
             'SSRT1':SSRT1,
             'SSRT2':SSRT2,
             'SSRT3':SSRT3,
             'SSRT4':SSRT4,
             'meanSSRT':meanSSRT,
             'Accuracy_Go':Accuracy_Go,
             'Accuracy_Stop':Accuracy_Stop,
             'Accuracy_Stop_1ststair':Accuracy_Stop_1ststair,
             'Accuracy_Stop_2ndstair':Accuracy_Stop_2ndstair,
             'Accuracy_Stop_3rdstair':Accuracy_Stop_3rdstair,
             'Accuracy_Stop_4thstair':Accuracy_Stop_4thstair,
             'meanGoRT':meanGoRT,
             'Percent_IncGo':Percent_IncGo,
             'Percent_MissGo':Percent_MissGo,
             'Percent_IncStop':Percent_IncStop,
             'Percent_FailStop':Percent_FailStop
            }

        #Create a DataFrame
        finaldata_collector = finaldata_collector.append(d, ignore_index=True)
        
out_file = os.path.join('/Shared/oleary/functional/UICL/BIDS/code/behav_data', 'SST-Behav-Data-ses1.csv')
finaldata_collector.to_csv(out_file, na_rep = 'n/a', index=False, sep='\t')